In [4]:
import pickle
import numpy as np
import pandas as pd
import sklearn

In [5]:
filename = 'OneHotEncodingList'
list_unquie_train=pickle.load( open(filename, 'rb'))

filename = 'LabelEncoder'
label_encoder=pickle.load(open(filename, 'rb'))

filename = 'Standardization'
sc=pickle.load( open(filename, 'rb'))

filename = 'Random_Forest_model'
rfc_classifier =pickle.load(open(filename, 'rb'))

In [7]:
filename = 'SVM_model'
svm_classifier =pickle.load(open(filename, 'rb'))

filename = 'XGB_model'
xgb_classififer =pickle.load(open(filename, 'rb'))


In [8]:
filename = 'knn_imputer'
knn_imputer =pickle.load(open(filename, 'rb'))

In [9]:
test_data=pd.read_csv("ms2-games-tas-test-v1.csv")

In [10]:
test_data.drop(['Subtitle','In-app Purchases','URL','Icon URL','ID','Name','Languages','Age Rating'],axis=1,inplace=True)

In [11]:
test_data['Original Release Date'] = pd.to_datetime(test_data['Original Release Date'],dayfirst=True).astype('datetime64[ns]').astype('int64')
test_data['Current Version Release Date'] = pd.to_datetime(test_data['Current Version Release Date'],dayfirst=True).astype('datetime64[ns]').astype('int64')

In [ ]:
test_data['Description'] = test_data['Description'].replace(len(test_data['Description']))
test_data['Description_length'] = [len(i) for i in test_data['Description']]
test_data['Description_word_length'] = [1 if i<= 500 else 2 if 500<i<=1000 else 3 for i in test_data['Description_length']]
test_data.drop(['Description', 'Description_length'], axis=1, inplace = True)


In [13]:
test_data['Size'] = test_data['Size']/(1024*1024)
test_data['Size'] = [1 if i<= 1 else 2 if 1<i<=10 else 3 for i in test_data['Size']]

# Handle Null values with imputers

In [14]:
test_data

,User Rating Count,Price,Description,Developer,Size,Primary Genre,Genres,Original Release Date,Current Version Release Date,Rate
0,1457.0,0.00,"""Divine Academy is an ancient strategy, an inc...",Qumaron,3,Games,"Games, Entertainment, Adventure, Strategy",1431907200000000000,1563321600000000000,High
1,75.0,4.99,Shogi is a Japanese version of chess.\nA playe...,Yoshikazu Kakinoki,3,Games,"Games, Strategy, Board",1232323200000000000,1443052800000000000,Intermediate
2,46733.0,0.00,"Epic heroes, monsters and action await in this...",Small Giant Games,3,Games,"Games, Strategy, Role Playing",1488326400000000000,1564617600000000000,High
3,2849.0,0.00,"Shadow Era is the full scale, cross-platform c...",Wulven Game Studios,3,Games,"Games, Strategy, Card, Entertainment",1365897600000000000,1561852800000000000,High
4,6.0,0.00,"""Command your army in epic battles in this new...",PLAYTOUCH,3,Games,"Games, Strategy, Entertainment, Board",1500854400000000000,1541116800000000000,Low
...,...,...,...,...,...,...,...,...,...,...
645,724.0,0.00,"""We hate to admit it, but we\u2019ve all been ...",Tapps Tecnologia da Informa\xe7\xe3o Ltda.,3,Games,"Games, Casual, Strategy",1429747200000000000,1453939200000000000,High
646,212.0,0.00,"Long long ago, maybe just when human starts to...",Fino Soft Inc.,3,Games,"Games, Casual, Strategy, Entertainment",1358553600000000000,1359072000000000000,High
647,59.0,0.00,Tanks of Battle: World War 2 combines both sim...,VascoGames,3,Games,"Games, Action, Strategy",1484784000000000000,1485561600000000000,High
648,5.0,0.00,"""Scopa and Scopone: choose two or four players...",Gsoftware srl semplificata,3,Games,"Games, Strategy, Entertainment, Card",1513555200000000000,1547856000000000000,High


In [15]:
def update_with_new_columns(data,list_all_unique,to_be_hot_encoded):
    # Add New Columns.
    updated_data = data
    updated_data = updated_data.reset_index(drop=True)
    for value in list_all_unique:
        new_column_arr = np.zeros(len(data))
        new_column_df = pd.Series(new_column_arr, name=value)
        updated_data = pd.concat([updated_data, new_column_df], axis=1)

    # Remove Old Columns
    for feature in to_be_hot_encoded:
        updated_data.drop(feature, axis=1, inplace=True)

    # Add Ones in the right columns.
    for feature in to_be_hot_encoded:
        for i in range(0, len(data)):
            cell = data[feature].iloc[i]
            if not pd.isnull(cell):
                valuesList = cell.split(',')
                for value in valuesList:
                    if value in updated_data: # Check if 'value' is seen in training operation.     #else: ignore it.
                        value_i = updated_data.columns.get_loc(value)
                        updated_data.iloc[i, value_i] = 1

    # Update.
    return updated_data

In [ ]:
teest=update_with_new_columns(test_data,list_unquie_train,['Developer','Genres','Primary Genre'])

In [ ]:
X_test=teest.drop(['Rate'],axis=1)
y_test=teest['Rate']

In [ ]:
teest = knn_imputer.transform(X_test)

In [ ]:
X_test_std = sc.transform(X_test)

In [ ]:
y_test= label_encoder.transform(y_test)

In [ ]:
from sklearn.metrics import classification_report,accuracy_score

y_pred = rfc_classifier.predict(X_test_std)

print(f"Accuracy of Test = {str(accuracy_score(y_test, y_pred))}\n")

Accuracy of Test = 0.7318963429401792

